<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/P1-NL2SQL/nl2sql_prompt_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">Learn by Doing LLM Projects</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Natural Language to SQL with LLMs</h2>
    <h3>Create a prompt to generate SQL with OpenAI Models</h3>
    by <b>Pere Martra</b>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

In one of the [first lessons in the course](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/nl2sql.ipynb), we did a Natural Language to SQL Generator using the OpenAI API. It works relatively fine, but the prompt was built in a basic style.

Now we are going to rebuild the prompt following the instructions in a paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853).


In [1]:
#First install the OpenAI library to acces gpt-3.5-turbo model.
!pip install openai==1.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
#You need an OpenAI key in order to use the OpenAI API https://platform.openai.com/account/api-keys

import openai
openai.api_key='your-opeai-api-key'

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to Maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853).

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
    create table employees(
        ID_Usr INT primary key,
        name VARCHAR);
    /*3 example rows
    select * from employees limit 3;
    ID_Usr    name
    1344      George StPierre
    2122      Jon jones
    1265      Anderson Silva
    */

    create table salary(
        ID_Usr INT,
        year DATE,
        salary FLOAT,
        foreign key (ID_Usr) references employees(ID_Usr));
    /*3 example rows
    select * from salary limit 3
    ID_Usr    date          salary
    1344      01/01/2023    61000
    1344      01/01/2022    60000
    1265      01/01/2023    55000
    */

    create table studies(
        ID_study INT,
        ID_Usr INT,
        educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */
        Institution VARCHAR,
        Years DATE,
        Speciality VARCHAR,
        primary key (ID_study, ID_Usr),
        foreign key(ID_Usr) references employees (ID_Usr));
    /*3 example rows
    select * from studies limit 3
    ID_Study ID_Usr educational_level Institution    Years       Speciality
    2782     1344   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
    2334     1344   5                 MIT            01/01/2023  Phd. Data Science.
    2782     2122   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
    */
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
 Question: How Many employes we have with a salary bigger than 50000?
SELECT COUNT(*) AS total_employees
FROM employees e
INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
WHERE s.salary > 50000;
 Question: Return the names of the three people who have had the highest salary increase in the last three years.
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
GROUP BY e.name
ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
LIMIT 3;
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("The name of the employee best paid", context_user))

SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("The name of the employee best paid", old_context_user))

This is your SQL:

SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;

Explanation: This SQL query joins the "employees" table with the "salary" table using the common field "ID_usr". It then orders the result by the salary in descending order and selects the name of the employee with the highest salary using the LIMIT clause.


In [9]:
#new
print(return_CCRMSQL("Return the Institution with a higher average salary", context_user))

SELECT s.Institution, AVG(s.salary) AS average_salary
FROM salary s
JOIN studies st ON s.ID_Usr = st.ID_Usr
GROUP BY s.Institution
ORDER BY average_salary DESC
LIMIT 1;


In [ ]:
#old
print(return_CCRMSQL("Return the Institution with a higher average salary", old_context_user))

This is your SQL:

SELECT Institution
FROM studies
JOIN employees ON studies.ID_usr = employees.ID_usr
JOIN salary ON studies.ID_usr = salary.ID_usr
GROUP BY Institution
ORDER BY AVG(salary) DESC
LIMIT 1;

Explanation: 
This SQL query joins the studies, employees, and salary tables using the ID_usr column. It then groups the data by Institution and calculates the average salary for each institution. The results are sorted in descending order by the average salary and the first row (highest average salary) is returned.


# Conclusion

I used this structure of prompt in several projects to generate SQL from natural language, and its result is really good. For simple Databases structure like the one in the notebook the differences in the results are really small, we must understand that GPT3.5-turbo is an excellent Large Language Model and well-trained for SQL generation.

As you can see in the second samples, as we increase the complexity of the question, we can find some differences in the SQL returned by the Model. It is not an easy task to decide which one is better or more efficient.

If you have access to a database with real data just try to obtain some SQL orders using a prompt with this structure and test by yourself how it works.